In [1]:
import os, folium, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import imageio.v3 as iio
import geopandas as gpd
from IPython.display import Image, display
import tensorflow as tf
%matplotlib inline

2023-05-22 09:19:23.422924: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 09:19:23.501348: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-22 09:19:24.491583: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2023-05-22 09:19:24.491760: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2023-05-22 09:19:24.515300: I tensorflow/c/logging.cc:34] Successfully opened dynamic l

In [2]:
from tensorflow.keras import layers, callbacks, backend, Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
labels= gpd.read_file('labels.geojson')
labels.head(2)

,id,Latitude,Longitude,Type,Source,Image_Folder,geometry
0,1,43.004434,-78.208900,Negative,Aeroplanes,train_images/43.00443446102501_-78.20890003579709,"POLYGON ((-78.20567 43.00681, -78.20567 43.002..."
1,6,42.033642,-97.719996,Negative,"White Spec (Seadoo, pivot irrigation, etc.)",train_images/42.03364226519898_-97.7199955063882,"POLYGON ((-97.71676 42.03605, -97.71676 42.031..."


In [4]:
labels_unique = labels[(labels.Type == 'Negative') | (labels.Type == 'CAFOs') | (labels.Type == 'WWTreatment')
           | (labels.Type == 'Landfills') | (labels.Type == 'RefineriesAndTerminals')
           | (labels.Type == 'ProcessingPlants') | (labels.Type == 'Mines')]

In [5]:
X = []
for im_path in labels_unique.Image_Folder.values:
     X.append(iio.imread(f'{im_path}/naip.png'))

In [6]:
X[0].shape

(720, 720, 4)

In [7]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels_unique.Type.values.reshape(labels_unique.Type.values.shape[0], 1))
mlb.classes_
y.shape

(14126, 7)

In [8]:
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.7, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=0.5, stratify=y_rest)


In [9]:
X_train = np.array(X_train)

In [10]:
type(X_train)

numpy.ndarray

In [11]:
X_val = np.array(X_val)

In [12]:
y_tra_orig = np.argwhere(y_train == 1.0)[:,1]
y_res_orig = np.argwhere(y_rest == 1.0)[:,1]

In [13]:
def invert_ecoding(encoded_labels, categories):
    categories = ['[unk]']+categories
    return np.take(categories, np.argwhere(encoded_labels == 1.0)[:,1])

In [15]:
def build_model():
    i = layers.Input(shape=(720,720, 3), dtype = tf.float32)
    i1 = layers.Resizing(
        224,
        224,
        interpolation='bilinear',
        crop_to_aspect_ratio=False
    )(i)
    out = tf.keras.applications.EfficientNetB0(include_top=True,
                      weights='imagenet'
                      )(i1)
    out = layers.Dense(32, activation='relu')(out)
    out = layers.Dense(7, activation='sigmoid')(out)
    model = Model(inputs=i, outputs=out)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten
from tensorflow.keras.models import Model\

#i = layers.Input(shape=(720,720, 4), dtype = tf.float32)
#i1 = layers.Resizing(
#        224,
#        224,
#        interpolation='bilinear',
#        crop_to_aspect_ratio=False
#)(i)

efficientnet_model = EfficientNetB0(include_top=False, weights=None, input_shape=(720,720, 4))
efficientnet_output = efficientnet_model.layers[-1].output

flatten_layer = Flatten()(efficientnet_output)

dense_layer = Dense(128, activation='relu')(flatten_layer)
output_layer = Dense(7, activation='sigmoid')(dense_layer)

model = Model(inputs=efficientnet_model.input, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/root/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/keras/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(
2023-05-22 09:23:52.384198: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 09:23:52.386499: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (NVIDIA RTX A4000 Laptop GPU)
2023-05-22 09:23:54.540031: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 1 (Intel(R) UHD Graphics)
2023-05-22 09:23:54.561970: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 720, 720, 4  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 720, 720, 4)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 720, 720, 4)  9           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 721, 721, 4)  0           ['normalization[0][0]']      

In [60]:
assert len(X_val) == len(y_val)

In [71]:
cb = [callbacks.EarlyStopping(patience=5), callbacks.ReduceLROnPlateau(patience=3)]
epochs = 10
batch_size = 4

history = model.fit(X_train,
         y_train,
         epochs=epochs,
         batch_size=batch_size,
         callbacks=cb,
         validation_data=(X_val, y_val))

In [2]:
!git add efficient_net.ipynb
!git commit -m 'first model'


On branch main
Your branch and 'origin/main' have diverged,
and have 1 and 1 different commits each, respectively.
  (use "git pull" to merge the remote branch into yours)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Data_/
	Methane_v0.ipynb
	labels.geojson
	train_images/

no changes added to commit (use "git add" and/or "git commit -a")
Username for 'https://github.com': ^C


In [ ]:
y_pred = model.predict(X_test)
y_pred[y_pred >= 0.5] = 1
y_pred[y_pred < 0.5] = 0

# score between y_test and y_pred